In [ ]:
# 🔁 Caricamento
import sys
sys.path.append("../src")

import torch
import torch.nn as nn
from torchvision import models
import matplotlib.pyplot as plt
import numpy as np
import cv2
from src.dataset_loader import get_dataloader
import os

# 📦 Dataloader
val_path = "/content/dataset_prepared/val" if "COLAB_GPU" in os.environ else "datasets/dataset_prepared/val"
dataset, dataloader = get_dataloader(data_dir=val_path, batch_size=32, shuffle=False)

# ⚙️ Modello
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load("results/histology_model.pth", map_location=device))
model = model.to(device)
model.eval()

# 🎯 Accuracy
correct, total = 0, 0
with torch.no_grad():
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

accuracy = 100 * correct / total
print(f"✅ Accuracy on validation set: {accuracy:.2f}%")
